In [10]:
queries = {
    'Q2': '''
SELECT s_acctbal, s_name, n_name, p_partkey, p_mfgr
    , s_address, s_phone, s_comment
FROM part, supplier, partsupp, nation, region
WHERE p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 30
    AND p_type LIKE '%STEEL'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND ps_supplycost = (
        SELECT MIN(ps_supplycost)
        FROM partsupp, supplier, nation, region
        WHERE p_partkey = ps_partkey
            AND s_suppkey = ps_suppkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'ASIA'
    )
ORDER BY s_acctbal DESC, n_name, s_name, p_partkey
LIMIT 100
    ''',
    'Q17': '''
SELECT sum(l_extendedprice) / 7.0 AS avg_yearly
FROM lineitem, part
WHERE p_partkey = l_partkey
    AND p_brand = 'Brand#44'
    AND p_container = 'WRAP PKG'
    AND l_quantity < (
        SELECT 0.2 * avg(l_quantity)
        FROM lineitem
        WHERE l_partkey = p_partkey
    )
    '''
}

In [39]:
import pymysql
import pymysql.cursors

class Connection:
    def __init__(self, host, port, user, password):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.conn = None
    
    def connect(self, database):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    passwd=self.password,
                                    db=database,
                                    charset='utf8')
    
    def select(self, sql):
        with self.conn.cursor() as cursor:
            cursor.execute(sql)
            result = cursor.fetchall()
        self.conn.commit()
        return str(result)
        
            
    def __del__(self):
        self.conn.close()

In [16]:
opt = Connection('127.0.0.1', 3306, 'test', '123456')
opt.connect('dbt3sf10')

opt.select("set global innodb_adaptive_hash_index = off;")
opt.select("set optimizer_switch='subquery_unnesting_with_window=on';")

In [27]:
opt = Connection('127.0.0.1', 3306, 'test', '123456')
opt.connect('dbt3sf10')

opt.select("set global innodb_adaptive_hash_index = off;")
result_of_queries = {}

for name in queries.keys():
    result_of_queries[name] = {}
    sql = queries[name]
    print (sql)
    
    opt.select("set optimizer_switch='subquery_unnesting_with_window=on';")
  
    result = %timeit -o -n 1 -r 10 opt.select(sql)
    result_of_queries[name]['unnest'] = (result.average, result.worst, result.best)
    
    opt.select("set optimizer_switch='subquery_unnesting_with_window=off';")

    result = %timeit -o -n 1 -r 10 opt.select(sql)
    result_of_queries[name]['non-unnest'] = (result.average, result.worst, result.best)
    
    print("%s speedup is: %f." % (name, result_of_queries[name]['non-unnest'][0] / result_of_queries[name]['unnest'][0]))



SELECT s_acctbal, s_name, n_name, p_partkey, p_mfgr
    , s_address, s_phone, s_comment
FROM part, supplier, partsupp, nation, region
WHERE p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 30
    AND p_type LIKE '%STEEL'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND ps_supplycost = (
        SELECT MIN(ps_supplycost)
        FROM partsupp, supplier, nation, region
        WHERE p_partkey = ps_partkey
            AND s_suppkey = ps_suppkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'ASIA'
    )
ORDER BY s_acctbal DESC, n_name, s_name, p_partkey
LIMIT 100
    
1.2 s ± 30.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.84 s ± 32.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Q2 speedup is: 1.535728.

SELECT sum(l_extendedprice) / 7.0 AS avg_yearly
FROM lineitem, part
WHERE p_partkey = l_partkey
    AND p_brand = 'Brand#44'
    

In [20]:
print(result_of_queries)

{'Q2': {'unnest': (1.2141587385998718, 1.232421185000021, 1.1703475929998604), 'non-unnest': (1.814908732799995, 1.8435294040000372, 1.780860814999869)}, 'Q17': {'unnest': (1.4744295098000293, 1.5379993710002964, 1.423272003999955), 'non-unnest': (6.9418928264001805, 7.01334007100013, 6.893790168000123)}}


# Regression of different data distribution

In [67]:
opt = Connection('11.160.211.136', 38472, 'test', '123456')
opt.connect('test')

opt.select("set global innodb_adaptive_hash_index = off;")
opt.select("set max_parallel_degree = 0;")
opt.select("drop table if exists part, partsupp;")
opt.select("""
CREATE TABLE part (
  p_partkey INT PRIMARY KEY
);
""")
opt.select("""
CREATE TABLE partsupp (
  ps_partkey INT,
  ps_supplycost DECIMAL(10,2),
  INDEX i1 (ps_partkey)
);
""")

base = 100000

#cursor=opt.cursor()
#v=cursor.callproc('insert_primary',[10,])
opt.select("call insert_primary(%d);" % base)
#opt.select("insert into part values(1)")

opt.select("analyze table part;")

#sf = 1, 2, 4, 8, 16)
selectivity = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#sf = [1, 2, 4, 8, 16]
sf = [10]

opt.select("set optimizer_switch='subquery_unnesting_with_window=on';")
warm_sql = """
SELECT COUNT(*) FROM part, partsupp 
WHERE p_partkey = ps_partkey 
  AND ps_supplycost >= (
    SELECT AVG(ps_supplycost) FROM partsupp WHERE p_partkey = ps_partkey);
"""
result = {}
for i in range(len(sf)):
    if i == 0:
        opt.select("call insert_secondary(%d, %d)" % (base, sf[i]))
    else:
        opt.select("call insert_secondary(%d, %d)" % (base, sf[i] - sf[i - 1]))
        
    opt.select("analyze table partsupp;")
    
    print("Current data sf: %d" % sf[i])
    
    sf_name = "%d" % sf[i]
    result[sf_name] = {}
    result[sf_name]["nest"] = []
    result[sf_name]["unnest"] = []
    
    once = True
    for sl in selectivity:
        sql = """
SELECT COUNT(*) FROM part, partsupp 
WHERE p_partkey = ps_partkey 
  AND ps_partkey <= %d * %f 
  AND ps_supplycost >= (
    SELECT AVG(ps_supplycost) FROM partsupp WHERE p_partkey = ps_partkey);
    """ % (base, sl)
        
        
        if once:
            opt.select("set optimizer_switch='subquery_unnesting_with_window=on';")
        
            tmp = %timeit -o -n 1 -r 5 opt.select(sql)
        
            result[sf_name]["unnest"].append(tmp.average)
        
            once = False
        
        
        
        opt.select("set optimizer_switch='subquery_unnesting_with_window=off';")
        
        #print(sql)
        #print(opt.select("EXPLAIN %s" % sql))
        
        tmp = %timeit -o -n 1 -r 5 opt.select(sql)
        
        result[sf_name]["nest"].append(tmp.average)
        
    print("nest: %s" % result[sf_name]["nest"])
    print("unnest: %s" % result[sf_name]["unnest"])
    


Current data sf: 10
4.59 s ± 39.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
3.09 s ± 6.64 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
6.14 s ± 26.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
9.26 s ± 46.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
12.4 s ± 42.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
15.5 s ± 7.99 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
18.7 s ± 50.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
21.8 s ± 21.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
24.8 s ± 38.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
28 s ± 46.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
31.1 s ± 27.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
nest: [3.093654219000018, 6.138807779600029, 9.26434011280071, 12.419903418400645, 15.482129662799707, 18.658413222799574, 21.771691373200156, 24.843864751199, 27.988789876999363, 31.058491639800195]
unnest: [4.58687551480034]
